## COG403 Final Report Code
#### Last Updated: April 14th, 2021
All code and figures included in this notebook are supporting materials for my final project entitled "Comparing Computational Models of Early Language Acquisition". The functions and driver code below show how a form of a Bayesian Network known as a Hidden Markov Model can be implemented to efficiently and effectively model parts of speech as found in conversation and written words. This Part of Speech tagging behavior is a dependency of Novel Verb Generation because to organize speech such that you can make correct generalizations about the usage of made-up words, incoming and noisy incoming speech must be reliably tagged with structural information to inform your current and past beliefs about sentence structure and available word choice.

1) Set-up Methods for modelling Parts of Speech using a Hidden Markov Model. 

In [9]:
# libraries
import os
import sys
import re
import collections as c
import numpy as np

def train_frequency(train_tags, train_words):
    "Return a frequency dictionary for each word and it's most probable tag"

    # prep collections
    counts = {}
    zipped = zip(train_tags, train_words)
    d = c.Counter(zipped)

    # Move through counter, collecting frequencies
    for word, tag in d:
        word_count = d[word, tag]

        # Word has yet to be seen
        if word not in counts.keys():
            counts[word] = {tag:word_count}

        # Already in return, ++count
        elif counts.get(word).get(tag):
            counts[word][tag] += 1

        # Update word count
        else:
            counts[word][tag] = word_count

    # Generate return dictionary of word:most frequent tag (the guess)
    output = {}
    for word, tag in counts.items():
        max_count = max(tag.keys(), key = (lambda key: tag[key]))
        output[word] = max_count

    return output

def make_test_dic(freq_dict, test_words):
    "Return an array of word:prediction elements for each word"
    zip = []
    keys = freq_dict.keys()

    # Fill zip with tuples of words to their most likely prediction given traiing
    for word in test_words:
        zip.append((word, freq_dict[word])) if word in keys else zip.append((word, 'VVD'))

    return zip


def setup_training_tables(train_words_and_tags, pos_dict):
    "Return pi, emission, transition, and word dictionaries for the Viterbi algorithm"

    # Initialize emission, transition and pi tables (len(pos_dict) == 91)
    num_pos, emissions_table, smol_prob = len(pos_dict), [], 0.000000001
    emissions_dict = c.defaultdict(lambda:len(emissions_dict))
    trans_table = np.ones(shape=(num_pos,num_pos))/10000
    pie_table = np.ones(num_pos)

    # For each file in training_files list
    for i in range(len(train_words_and_tags)):
        ancestor = [".", "PUN"]                # Junk val on first iter
        train_file_i = train_words_and_tags[i] # [[word, POS], [word, POS], ...]

        # for each pair, update E,T,PI tables and update prev value for next loop
        for i in range(len(train_file_i)):
            word = train_file_i[i][0]
            pos = train_file_i[i][1]
            
            # Fill Emission Table if it isnt already
            emissions_dict[word]
            if len(emissions_table) < len(emissions_dict):
                emissions_table.append([smol_prob]*num_pos)
            emissions_table[emissions_dict[word]][pos_dict[pos]] += 1

            # Fill Transition table
            trans_table[pos_dict[ancestor[1]]][pos_dict[pos]] += 1

            # Fill Pi Table using pointer to last seen elem
            if ancestor[0] == "." and ancestor[1] == "PUN":
                pie_table[pos_dict[pos]] += 1

            # Update ancestor pointer to curr pair for next iter
            ancestor = train_file_i[i]

    # Normalize by Row and by Column
    emissions_table = np.array(emissions_table)
    emissions_table = emissions_table/emissions_table.sum(axis=0, keepdims=1)
    trans_table = trans_table/trans_table.sum(axis=1, keepdims=1)
    pie_table = pie_table / pie_table.sum(keepdims=1)

    # Return tables for Viterbi
    return pie_table, trans_table, emissions_table, emissions_dict

def viterbi(training_pairs, test_words, pos):
    "Apply Viterbi to test_words and return a path/probs and word list"

    # Set up Pie, Emission, and Transition tables from training list data
    pie, trans, emis, emis_dict = setup_training_tables(training_pairs, pos)
    n_pos = len(pos)

    # Init empty prob tables
    prev_prob, curr_prob = np.zeros(shape=(n_pos)), np.zeros(shape=(n_pos))
    prev_path, curr_path = np.empty(shape=(n_pos), dtype='object'), np.empty(shape=(n_pos), dtype='object')

    # Fill prev_prob and prev_path tables from emission dict
    for i in range(n_pos):
        prev_prob[i] = pie[i] * emis[emis_dict[test_words[0]]][i]
        prev_path[i] = str(i) + ','

    # Take len(test_words) observations and calculate trellis
    for t in range(1, len(test_words)):
        curr_prob = np.zeros(shape=(n_pos))
        curr_path = np.empty(shape=(n_pos), dtype='object')
        
        if t%5000 == 0:
            print(f"Tagging...Taken {t} observations")

        # get argmax, calc emission
        for s in range(n_pos):
            i = np.argmax(prev_prob * trans[:,s])
            if emis_dict[test_words[t]] >= len(emis):
                yuh = 1
            else:
                yuh = emis[emis_dict[test_words[t]],s]

            curr_prob[s] = prev_prob[i] * trans[i,s] * yuh
            curr_path[s] = prev_path[i] + str(s) + ','

        # normalize every step to avoid crashing into 0
        curr_prob = curr_prob/curr_prob.sum()

        # copy over path_curr to path_prev
        prev_path = np.copy(curr_path)
        prev_prob = np.copy(curr_prob)

    return curr_prob, curr_path

def tag(training_list, test_file, output_file):
    "Tag the words from the untagged input file and write them into the output file."

    # init storage
    train_words_and_tags, test_words = [], []

    # Setup POS dictionary
    all_pos = [ "AJ0-AV0", "AJ0-NN1", "AJ0-VVD", "AJ0-VVG", "AJ0-VVN", "AV0-AJ0", "AVP-PRP", "AVQ-CJS", "CJS-AVQ", "CJS-PRP", 
                 "CJT-DT0", "CRD-PNI", "DT0-CJT", "NN1-AJ0", "NN1-NP0", "NN1-VVB", "NN1-VVG", "NN2-VVZ", "NP0-NN1", "PNI-CRD", 
                 "PRP-AVP", "PRP-CJS", "VVB-NN1", "VVD-AJ0", "VVD-VVN", "VVG-AJ0", "VVG-NN1", "VVN-AJ0", "VVN-VVD", "VVZ-NN2", 
                 "AJ0", "AJC", "AJS", "AT0", "AV0", "AVP", "AVQ", "CJC", "CJS", "CJT", "CRD", "DPS", "DT0", "DTQ", "EX0", "ITJ", 
                 "NN0", "NN1", "NN2", "NP0", "ORD", "PNI", "PNP", "PNQ", "PNX", "POS", "PRF", "PRP", "PUL", "PUN", "PUQ", "PUR", 
                 "TO0", "UNC", "VBB", "VBD", "VBG", "VBI", "VBN", "VBZ", "VDB", "VDD", "VDG", "VDI", "VDN", "VDZ", "VHB", "VHD", 
                 "VHG", "VHI", "VHN", "VHZ", "VM0", "VVB", "VVD", "VVG", "VVI", "VVN", "VVZ", "XX0", "ZZ0"]

    # Fill POS dict ({'AJ0-AVO': 0, 'AJ0-NN1: 1, ...})
    pos = {all_pos[i]:i for i in range(len(all_pos))}

    # Import training dataset files
    print("Importing training dataset")
    for file in training_list:
        this_file = []
        train = open(file, 'r')
        for line in train.readlines():
            zipd = [term.rstrip('\n') for term in re.split(' : ', line)] # ['Word', 'POS']
            this_file.append(zipd)
        train_words_and_tags.append(this_file) # list of len(training_list) sublists, each with [Word, POS] pair elems
        train.close()

    # Import testing dataset
    print("Importing testing dataset")
    test = open(test_file)
    for line in test.readlines():
        word = line.strip('\n')
        test_words.append(word)

    # Use Viterbi to get path and POS predictions
    print("Starting HMM Part of Speech tagging algorithm")
    v_prob, v_path = viterbi(train_words_and_tags, test_words, pos)
    print("Finished HMM Part of Speech tagging algorithm")

    # Write output
    print("Writing to output. About to exit.")
    f = open(output_file, "w")
    ind = np.argmax(v_prob)
    sol_pos = v_path[ind].split(',')
    for i in range(len(test_words)):
        line = f"{test_words[i]} : {all_pos[int(sol_pos[i])]}\n"
        f.write(line)
    f.close()

Testing the Markov Model Part of Speech Tagger:

In [10]:
# Run the tagger function.
print("Starting the tagging process.")

training_list = ['training1.txt']
# training_list = ['training123.txt'] Uncomment this to test on 3x more words
print("Training files: " + str(training_list))

test_file = 'test1.txt'
# test_file = 'test123.txt' Uncomment this to test on 3x more words
print("Test file: " + test_file)

output_file = 'output.txt'
print("Ouptut file: " + output_file)

# creates output.txt
tag(training_list, test_file, output_file)

Starting the tagging process.
Training files: ['training1.txt']
Test file: test1.txt
Ouptut file: output.txt
Importing training dataset
Importing testing dataset
Starting HMM Part of Speech tagging algorithm
Tagging...Taken 5000 observations
Tagging...Taken 10000 observations
Tagging...Taken 15000 observations
Tagging...Taken 20000 observations
Tagging...Taken 25000 observations
Tagging...Taken 30000 observations
Tagging...Taken 35000 observations
Tagging...Taken 40000 observations
Tagging...Taken 45000 observations
Finished HMM Part of Speech tagging algorithm
Writing to output. About to exit.


Check this against the solutions:

In [11]:
with open("output.txt", "r") as output_file, \
     open("training1.txt", "r") as solution_file:

    # Each word is on a separate line in each file.
    output = output_file.readlines()
    solution = solution_file.readlines()
    total_matches = 0

    # generate the report
    for index in range(len(output)):
        if output[index] != solution[index]:
            print(f"Line {index + 1}: "
                  f"expected <{output[index].strip()}> "
                  f"but got <{solution[index].strip()}>")
        else:
            total_matches = total_matches + 1

    # Add stats at the end of the results file.
    print(f"Total words seen: {len(output)}.")
    print(f"Total matches: {total_matches}.")
    print(f"Percentage Accuracy?: {total_matches/len(output)}")

Line 11: expected <before : CJS> but got <before : PRP>
Line 38: expected <New : NP0> but got <New : AJ0>
Line 57: expected <straight : AV0-AJ0> but got <straight : AV0>
Line 58: expected <on : PRP> but got <on : AVP>
Line 165: expected <off : PRP> but got <off : PRP-AVP>
Line 226: expected <New : NP0> but got <New : AJ0>
Line 249: expected <that : CJT> but got <that : DT0>
Line 261: expected <'ve : VHB> but got <'ve : VHI>
Line 528: expected <over : PRP-AVP> but got <over : AV0>
Line 537: expected <little : AV0> but got <little : AV0-AJ0>
Line 596: expected <on : PRP> but got <on : PRP-AVP>
Line 800: expected <as : CJS> but got <as : CJS-PRP>
Line 811: expected <considered : VVD> but got <considered : VVN>
Line 815: expected <All : AV0> but got <All : DT0>
Line 871: expected <made : VVD> but got <made : VVN-VVD>
Line 880: expected <lift : NN1> but got <lift : NN1-VVB>
Line 888: expected <New : NP0> but got <New : AJ0>
Line 946: expected <her : DPS> but got <her : PNP>
Line 963: expect

As shown by this implementation, and the accuracy of ~97.36%, not only is Bayesian Part of Speech modelling a likely possibility for human cognition, it is also statistically optimal and computationally efficient. Even if the mind does not implement an event/probability based model for tracking and destructuring incoming syntactic information, this shows that a combination of heuristics could come to be performant at that task. This is an important finding for the conclusions about Bayesian linguistic models vs. classical backpropogation models as discussed in the report, because the strength of this HMM lends strength to the idea that the mind uses Bayesian or Dependent Probabilistic relationships to model abstract information.